# HOMEWORK 07

In [18]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tqdm import tqdm
import datetime

## Prepare Dataset

In [19]:
# Load MNIST dataset
def load_data():
    (train_ds, test_ds), ds_info = tfds.load('mnist', split=['train', 'test'], as_supervised=True, with_info=True)
    return (train_ds, test_ds), ds_info

In [20]:
def new_target_fnc(ds, window_size):
  l = list()
  for i, elem in enumerate(ds):
    if (i % window_size) == 0:
      l.append(int(elem[1]))
    else:
      if (i % 2) == 0:
        l.append(int(l[i-1] + elem[1]))
      else:
        l.append(int(l[i-1] - elem[1]))
  return l

def preprocess(data, batch_size, window_size):
  # replacing the target with a new target
  new_targets = new_target_fnc(data, window_size)
  new_targets = tf.data.Dataset.from_tensor_slices(new_targets)
  data = tf.data.Dataset.zip((data, new_targets))
  data = data.map(lambda img, new_target: (img[0], new_target))

  # convert (uint8 to float32)
  data = data.map(lambda img, target: (tf.cast(img, tf.float32), target))
  data = data.map(lambda img, target: ((img/128.)-1., target))
  # create one-hot targets (39 classes from -18 to 18)
  data = data.map(lambda img, target: (img, tf.one_hot(target, depth=39)))
  
  data = tf.data.Dataset.window(data, window_size)
  # caching transformation on dataset 
  data = data.cache() 

  # data = data.shuffle(1000) # Does it shuffle whole points or windows?
  data = data.batch(batch_size)
  
  data = data.prefetch(tf.data.AUTOTUNE)
  return data

In [21]:
class CNN(tf.keras.Model):
  def __init__(self, optimizer, loss_function, input_shape):
    super().__init__()
    self.conv1 = tf.keras.layers.Conv2D(28, 3, activation='relu', padding='same', input_shape=input_shape[2:])
    self.conv2 = tf.keras.layers.Conv2D(28, 3, activation='relu', padding='same', input_shape=input_shape[2:])
    self.pooling1 = tf.keras.layers.AveragePooling2D()
    self.conv3 = tf.keras.layers.Conv2D(14, 3, activation='relu', padding='same', input_shape=input_shape[2:])
    self.conv4 = tf.keras.layers.Conv2D(14, 3, activation='relu', padding='same', input_shape=input_shape[2:])
    self.pooling2 = tf.keras.layers.AveragePooling2D()
    self.conv5 = tf.keras.layers.Conv2D(7, 3, activation='relu', padding='same', input_shape=input_shape[2:])
    self.conv6 = tf.keras.layers.Conv2D(7, 3, activation='relu', padding='same', input_shape=input_shape[2:])
    self.globalpooling = tf.keras.layers.GlobalAvgPool2D()
    self.out = tf.keras.layers.Dense(10, activation='softmax')

    self.optimizer = optimizer
    self.loss_function = loss_function

    self.metrics_list = [
        tf.keras.metrics.CategoricalAccuracy(name="accuracy"),
        tf.keras.metrics.Mean(name="loss")
    ]

  def call(self, x, training=False):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.pooling1(x)
    x = self.conv3(x)
    x = self.conv4(x)
    x = self.pooling2(x)
    x = self.conv5(x)
    x = self.conv6(x)
    x = self.globalpooling(x)
    x = self.out(x)
    return x

  # reset all metrics
  def reset_metrics(self):
      for metric in self.metrics:
          metric.reset_states()

  @tf.function
  def train_step(self, data):
      image, label = data

      with tf.GradientTape() as tape:
          prediction = self(image, training = True)
          loss = self.loss_function(label, prediction)

      gradients = tape.gradient(loss, self.trainable_variables)
      self.optimizer.apply_gradients(zip(gradients,self.trainable_variables))
      self.metrics[0].update_state(label, prediction)
      self.metrics[1].update_state(loss)

  @tf.function
  def test_step(self, data):
      image, label = data
      prediction = self(image, training = False)
      loss = self.loss_function(label, prediction)
      self.metrics[0].update_state(label, prediction)
      self.metrics[1].update_state(loss)


def training_loop(model, train_ds, test_ds, epochs, train_summary_writer, test_summary_writer, save_path):
    for epoch in range (epochs):
        model.reset_metrics()

        for data in tqdm(train_ds, position=0, leave=True):
            model.train_step(data)

        with train_summary_writer.as_default():
            tf.summary.scalar(model.metrics[0].name, model.metrics[0].result(), step=epoch)
            tf.summary.scalar(model.metrics[1].name, model.metrics[1].result(), step=epoch)
        
        print("Epoch: ", epoch+1)
        print("Loss: ", model.metrics[1].result().numpy(), "Accuracy: ", model.metrics[0].result().numpy(), "(Train)")
        model.reset_metrics()

        for data in test_ds:
            model.test_step(data)

        with test_summary_writer.as_default():
            tf.summary.scalar(model.metrics[0].name, model.metrics[0].result(), step=epoch)
            tf.summary.scalar(model.metrics[1].name, model.metrics[1].result(), step=epoch)

        print("Loss: ", model.metrics[1].result().numpy(), "Accuracy: ", model.metrics[0].result().numpy(), "(Test)")
    
    model.save_weights(save_path)

In [22]:
batch_size = 32
window_size = 4
(train_ds,test_ds), ds_info = load_data()
train_ds = preprocess(train_ds, batch_size, window_size)
test_ds = preprocess(test_ds, batch_size, window_size)

optimizer = tf.keras.optimizers.Adam()
loss_function = tf.keras.losses.CategoricalCrossentropy()
cnn = CNN(optimizer=optimizer, loss_function=loss_function, input_shape=(batch_size, window_size, 28, 28, 1))
epochs = 10

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
save_path = f"models/{current_time}"
train_log_path = f"logs/{current_time}/train"
test_log_path = f"logs/{current_time}/test"
train_summary_writer = tf.summary.create_file_writer(train_log_path)
test_summary_writer = tf.summary.create_file_writer(test_log_path)
training_loop(cnn, train_ds, test_ds, epochs, train_summary_writer, test_summary_writer, save_path)

  0%|          | 0/469 [00:00<?, ?it/s]


TypeError: in user code:

    File "/var/folders/wt/xy7579_50v90hx4jnj0017rc0000gn/T/ipykernel_35194/4221311034.py", line 46, in train_step  *
        prediction = self(image, training = True)
    File "/opt/miniconda3/envs/scipy/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler  **
        raise e.with_traceback(filtered_tb) from None

    TypeError: Exception encountered when calling layer "cnn_3" (type CNN).
    
    in user code:
    
        File "/var/folders/wt/xy7579_50v90hx4jnj0017rc0000gn/T/ipykernel_35194/4221311034.py", line 24, in call  *
            x = self.conv1(x)
        File "/opt/miniconda3/envs/scipy/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler  **
            raise e.with_traceback(filtered_tb) from None
        File "/opt/miniconda3/envs/scipy/lib/python3.10/site-packages/keras/engine/input_spec.py", line 197, in assert_input_compatibility
            raise TypeError(f'Inputs to a la